In [12]:
import subprocess

def get_branches():
    # Get the list of all branches
    result = subprocess.run(["git", "branch", "-a"], capture_output=True, text=True)
    branches = result.stdout.strip().split('\n')

    # Filter and clean up branch names
    cleaned_branches = []
    for branch in branches:
        # Remove leading asterisk and whitespace, and skip remote branches
        cleaned_branch = branch.replace('*', '').strip().split('/')[-1]
        if cleaned_branch not in cleaned_branches and not branch.startswith('remotes/'):
            cleaned_branches.append(cleaned_branch)

    return cleaned_branches

# Get the list of branches
branches = get_branches()


In [13]:
import json
import os

def score_bracket(bracket, answers):
    score = 0
    points_per_round = {1: 1, 2: 2, 3: 4, 4: 8, 5: 16, 6: 32}  # Points for each round

    with open(bracket) as fb, open(answers) as ak:
        answer_key = json.load(ak)
        user_bracket = json.load(fb)
        for round_num, round_name in enumerate(answer_key, start=1):
            for game_id, game in answer_key[round_name].items():
                if game_id in user_bracket[round_name]:
                    # Compare the winners in the same game of both brackets
                    if game["winner"] == user_bracket[round_name][game_id]["winner"]:
                        score += points_per_round[round_num]
    
    return score

In [14]:
branches = get_branches()
scores = {}

In [15]:
branches

['Chasem2020', 'Titodreamer', 'cdmoseley', 'main', 'sakar123', 'tomshafer']

In [16]:
for branch in branches:
    if branch == 'tomshafer':
        continue  # Skip the main branch
    
    # Checkout 'final_bracket.json' from the current branch
    subprocess.run(["git", "checkout", branch, "--", "final_bracket.json"], check=True)
    
    # Call your scoring function
    score = score_bracket("final_bracket.json", "answer_key.json")
    scores[branch] = score

    # Restore the original state of 'final_bracket.json' if needed
    subprocess.run(["git", "checkout", "main", "--", "final_bracket.json"], check=True)


In [17]:
scores

{'Chasem2020': 174,
 'Titodreamer': 203,
 'cdmoseley': 107,
 'main': 107,
 'sakar123': 220}